In [ ]:
import os
from PIL import Image
from torchvision import transforms
import xml.etree.ElementTree as ET

# Define the transformation
transform = transforms.Compose([
    transforms.Resize((320, 320)),  # Resize image to 320x320
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ImageNet normalization
])

def transform_and_save_images(input_dir, output_dir, annotation_dir, annotation_output_dir):
    # Ensure the output directories exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    if not os.path.exists(annotation_output_dir):
        os.makedirs(annotation_output_dir)

    # Loop over all images in the input directory
    for img_name in os.listdir(input_dir):
        if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(input_dir, img_name)
            annotation_path = os.path.join(annotation_dir, os.path.splitext(img_name)[0] + '.xml')

            # Check if the output already exists
            output_img_path = os.path.join(output_dir, img_name)
            output_annotation_path = os.path.join(annotation_output_dir, os.path.splitext(img_name)[0] + '.xml')

            if os.path.exists(output_img_path):
                print(f"Skipped {img_name} (already exists)")
                continue

            try:
                # Load and transform the image
                image = Image.open(img_path).convert("RGB")
                transformed_image = transform(image)
                transformed_image = transforms.ToPILImage()(transformed_image)
                transformed_image.save(output_img_path)
                print(f"Transformed and saved: {img_name}")

                # Process the corresponding annotation if it exists
                if os.path.exists(annotation_path):
                    tree = ET.parse(annotation_path)
                    root = tree.getroot()

                    # Scale bounding boxes to the new image size (320x320)
                    width, height = image.size
                    new_width, new_height = 320, 320
                    scale_x = new_width / width
                    scale_y = new_height / height

                    for obj in root.findall("object"):
                        bndbox = obj.find("bndbox")
                        xmin = int(bndbox.find("xmin").text) * scale_x
                        ymin = int(bndbox.find("ymin").text) * scale_y
                        xmax = int(bndbox.find("xmax").text) * scale_x
                        ymax = int(bndbox.find("ymax").text) * scale_y

                        # Check bounding box size (skip if too small)
                        if (xmax - xmin) < 10 or (ymax - ymin) < 10:
                            root.remove(obj)

                        else:
                            # Update bounding box values
                            bndbox.find("xmin").text = str(int(xmin))
                            bndbox.find("ymin").text = str(int(ymin))
                            bndbox.find("xmax").text = str(int(xmax))
                            bndbox.find("ymax").text = str(int(ymax))

                    # Save the updated annotation if valid objects remain
                    if root.findall("object"):
                        tree.write(output_annotation_path)
                        print(f"Transformed and saved annotation: {os.path.splitext(img_name)[0]}.xml")
                    else:
                        print(f"Skipped annotation for {img_name} (no valid objects)")

            except Exception as e:
                print(f"Failed to process {img_name}: {e}")

# Input and output directories for training
train_image_dir = "datasets/all/train/images"
train_annotation_dir = "datasets/all/train/annotations"
train_output_image_dir = "datasets/all/train/images_transformed"
train_output_annotation_dir = "datasets/all/train/annotations_transformed"

transform_and_save_images(
    train_image_dir, train_output_image_dir, train_annotation_dir, train_output_annotation_dir
)

# Input and output directories for validation
val_image_dir = "datasets/all/val/images"
val_annotation_dir = "datasets/all/val/annotations"
val_output_image_dir = "datasets/all/val/images_transformed"
val_output_annotation_dir = "datasets/all/val/annotations_transformed"

transform_and_save_images(
    val_image_dir, val_output_image_dir, val_annotation_dir, val_output_annotation_dir
)


In [ ]:
import os
import xml.etree.ElementTree as ET

def check_empty_annotations(annotation_dir):
    empty_annotation_files = []
    for annotation_file in os.listdir(annotation_dir):
        if annotation_file.endswith(".xml"):
            annotation_path = os.path.join(annotation_dir, annotation_file)
            try:
                tree = ET.parse(annotation_path)
                root = tree.getroot()
                objects = root.findall('object')
                for obj in objects:
                    bndbox = obj.find('bndbox')
                    if bndbox is not None:
                        xmin = float(bndbox.find('xmin').text)
                        ymin = float(bndbox.find('ymin').text)
                        xmax = float(bndbox.find('xmax').text)
                        ymax = float(bndbox.find('ymax').text)
                        if xmin == 0.0 and ymin == 0.0 and xmax == 0.0 and ymax == 0.0:
                            print(f"Invalid bounding box in file {annotation_file}: {xmin, ymin, xmax, ymax}")
                            empty_annotation_files.append(annotation_file)
                            break  # No need to check further for this annotation file
                if len(objects) == 0:
                    empty_annotation_files.append(annotation_file)
            except ET.ParseError:
                print(f"Error parsing XML file: {annotation_file}")
                empty_annotation_files.append(annotation_file)  # Consider it empty if parsing fails
    return empty_annotation_files


# Check for empty annotations in train and validation sets
train_annotation_dir = "/content/drive/MyDrive/all/train/annotations_transformed"
val_annotation_dir = "/content/drive/MyDrive/all/val/annotations_transformed"

empty_train_annotations = check_empty_annotations(train_annotation_dir)
empty_val_annotations = check_empty_annotations(val_annotation_dir)

print("Empty annotations in train set:", empty_train_annotations)
print("Empty annotations in validation set:", empty_val_annotations)